# Solution Idea
Generate input directions that the tail follows. Use this output again as an input. Repeat 9 times.

In [1]:
import numpy as np

In [2]:
def move_head(row, col, direction):
    if direction == 'R':
        col += 1
    elif direction == 'L':
        col -= 1
    elif direction == 'D':
        row += 1
    elif direction == 'U':
        row -= 1
    elif direction == 'DR':
        row += 1
        col += 1
    elif direction == 'DL':
        row += 1
        col -= 1
    elif direction == 'UR':
        row -= 1
        col += 1
    elif direction == 'UL':
        row -= 1
        col -= 1
    return row, col

In [3]:
def move_tail(row_head, col_head, row_tail, col_tail):
    row_delta = row_head - row_tail
    col_delta = col_head - col_tail
    if abs(row_delta) <= 1 and abs(col_delta) <= 1:
        return row_tail, col_tail
    elif row_delta > 1:
        if col_delta >= 1:
            return row_tail + 1, col_tail + 1
        elif col_delta <= -1:
            return row_tail + 1, col_tail - 1
        else:
            return row_tail + 1, col_tail
    elif row_delta < -1:
        if col_delta >= 1:
            return row_tail - 1, col_tail + 1
        elif col_delta <= -1:
            return row_tail - 1, col_tail - 1
        else:
            return row_tail - 1, col_tail
    elif col_delta > 1:
        if row_delta >= 1:
            return row_tail + 1, col_tail + 1
        elif row_delta <= -1:
            return row_tail - 1, col_tail + 1
        else:
            return row_tail, col_tail + 1
    elif col_delta < -1:
        if row_delta >= 1:
            return row_tail + 1, col_tail - 1
        elif row_delta <= -1:
            return row_tail - 1, col_tail - 1
        else:
            return row_tail, col_tail - 1

In [4]:
def create_input_for_next_segment(row_head, col_head, inputs):
    row_tail = row_head
    col_tail = col_head
    m_tail = np.zeros((row_head * 2, col_head * 2))
    m_tail[row_tail, col_tail] = 1
    moves = []
    moved = True
    for l in inputs:
        direction, repetition = l.split()
        for i in range(int(repetition)):
            if moved:
                row_tail_old = row_tail
                col_tail_old = col_tail
            moved = True
            try:
                row_head, col_head = move_head(row_head, col_head, direction)
                row_tail, col_tail = move_tail(row_head, col_head, row_tail, col_tail)
            except Exception as ex:
                print(ex)
            #             print(f'row_head {row_head} col_head {col_head} row_tail {row_tail} col_tail {col_tail} row_tail_old {row_tail_old} col_tail_old {col_tail_old}')
            if row_tail_old < row_tail:
                if col_tail_old < col_tail:
                    moves.append(f'DR 1')
                elif col_tail_old > col_tail:
                    moves.append(f'DL 1')
                else:
                    moves.append(f'D 1')
            elif row_tail_old > row_tail:
                if col_tail_old < col_tail:
                    moves.append(f'UR 1')
                elif col_tail_old > col_tail:
                    moves.append(f'UL 1')
                else:
                    moves.append(f'U 1')
            elif col_tail_old < col_tail:
                moves.append(f'R 1')
            elif col_tail_old > col_tail:
                moves.append(f'L 1')
            else:
                moved = False

            m_tail[row_tail, col_tail] = 1
    return moves, m_tail, row_tail, col_tail

In [5]:
def print_matrix(matrix, title):
    print(title)
    for x in matrix.astype(int):
        print('|'.join(map(str, x)))
    print('----------------------------------------------------------')

In [6]:
def calculate_field(path, do_print=False, dim_row=500, dim_col=500, command_lines=None):
    with open(path) as f:
        lines = f.read().splitlines()
    # initialize big enough playing field, positioning init in the center
    row_head = dim_row
    col_head = dim_col
    m_last = np.zeros((row_head * 2, col_head * 2))
    
    if command_lines:
#         show field after n commands (useful for debugging)
        print(f'Only using a subset of [{command_lines}] commands')
        used_commands = lines[:command_lines]
    else:
        used_commands = lines
    moves, m_tail, last_row, last_col = create_input_for_next_segment(row_head, col_head, used_commands)
    m_last[last_row, last_col] = 1
    for i in range(2, 10):
        moves, m_tail,  last_row, last_col = create_input_for_next_segment(row_head, col_head, moves)
        m_last[last_row, last_col] = i
        
    print(f'Tainted fields: {np.sum(m_tail)}')
    if do_print:
        if command_lines:
            print_matrix(m_last, f'Position after [{command_lines}] commands')
        else:
            print_matrix(m_last, f'Position after all commands')
    if do_print:
        print_matrix(m_tail, 'Tainted fields')

In [7]:
calculate_field('test_data2.txt', True, 15, 15)

Tainted fields: 36.0
Position after all commands
0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|1|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|2|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|3|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|4|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|5|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|6|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|7|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|8|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|9|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0

In [8]:
calculate_field('test_data2.txt', True, 15, 15,2)

Only using a subset of [2] commands
Tainted fields: 1.0
Position after [2] commands
0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0|0|0|0|0|0|0|0
0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|2|0|0|0|0|0|0|0|0|0
0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|3|0|0|0|0|0|0|0|0|0
0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|5|4|0|0|0|0|0|0|0|0|0
0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|6|0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|7|0|0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|8|0|0|0|0|0|0|0|0|0|0|0|0|0
0|0|0|0|0|0|0|0|

In [9]:
calculate_field('data.txt')

Tainted fields: 2536.0
